# 📗 Part 3 v3 완전판

**Priority 1:** Ensemble+Statistical Test, n_iter=200  

**Priority 2:** Cumulative Gains, 시각화, CV 임계값  

**Priority 3:** Winsorizer, Calibration, Learning Curve

## 0. 환경 설정

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import platform
if platform.system() == 'Darwin': plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows': plt.rc('font', family='Malgun Gothic')
else: plt.rc('font', family='NanumGothic')
plt.rc('axes', unicode_minus=False)
from tqdm.auto import tqdm




from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, cross_val_predict, RandomizedSearchCV, learning_curve
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import confusion_matrix, roc_auc_score, average_precision_score, precision_recall_curve, fbeta_score, make_scorer
from sklearn.calibration import calibration_curve
from imblearn.over_sampling import SMOTE, BorderlineSMOTE
from imblearn.combine import SMOTETomek
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier,RandomForestClassifier, ExtraTreesClassifier
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
from scipy.stats import wilcoxon
import joblib, os
from datetime import datetime
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
print('✅ 환경 설정 완료')

✅ 환경 설정 완료


In [2]:
DATA_DIR = '../data'
PROCESSED_DIR = os.path.join(DATA_DIR, 'processed')
os.makedirs(PROCESSED_DIR, exist_ok=True)
INPUT_FILE = os.path.join(DATA_DIR, 'features', 'domain_based_features_완전판.csv')
OUTPUT_PREFIX = '발표_Part3_v3'

## 1. 3-Way Split (Train 60% / Val 20% / Test 20%)

### 🛑 모델링 전략 핵심 변경

#### 1. 변수 제거: `'이해관계자_불신지수'` 🚫
* **사유:** 부도 발생과 동시에 나타나는 **후행성 지표(Data Leakage)**입니다.
* **결정:** 이를 제외하여 "이미 망한 기업 탐지"가 아닌, **"망할 징후를 미리 포착(조기 경보)"**하는 순수 재무 기반 모델로 전환했습니다.

#### 2. 데이터 분할 전략: 3-Way Split (60% : 20% : 20%) 📊
* **Train (60%):** 불균형한 데이터(1.5% 부도율)에서 소수 클래스를 충분히 학습하기 위함.
* **Valid (20%):** 모델 튜닝(AutoML) 및 최적 임계값(Threshold) 결정을 위한 검증용.
* **Test (20%):** 학습 과정에 전혀 관여하지 않은 **완전한 미지(Unseen) 데이터**로 최종 성능을 객관적으로 평가.

In [3]:
df = pd.read_csv(INPUT_FILE, encoding='utf-8')
TARGET_COL = '모형개발용Performance(향후1년내부도여부)'

X = df.drop(columns=[TARGET_COL])

# -------------------------------------------------------
# [수정] 논리적 오류 방지를 위해 '이해관계자_불신지수' 제거
# -------------------------------------------------------
drop_cols = ['이해관계자_불신지수']
existing_drop = [c for c in drop_cols if c in X.columns]
if existing_drop:
    X = X.drop(columns=existing_drop)
    print(f"🚫 제거된 변수: {existing_drop}")
# -------------------------------------------------------

y = df[TARGET_COL]
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=RANDOM_STATE)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, stratify=y_temp, random_state=RANDOM_STATE)
print(f'Train: {len(X_train):,} | Val: {len(X_val):,} | Test: {len(X_test):,}')
print('⚠️ Test는 최종 평가까지 사용 안 함!')

🚫 제거된 변수: ['이해관계자_불신지수']
Train: 30,063 | Val: 10,021 | Test: 10,021
⚠️ Test는 최종 평가까지 사용 안 함!


## 2. 전처리 + ⭐ Winsorizer 실험 (Priority 3)

In [4]:
from sklearn.base import BaseEstimator, TransformerMixin

class InfiniteHandler(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None): return self
    def transform(self, X): return X.replace([np.inf, -np.inf], np.nan)

class LogTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, eps=1e-10): self.eps = eps
    def fit(self, X, y=None): return self
    def transform(self, X):
        X_c = X.copy()
        for c in X_c.columns:
            if (X_c[c] >= 0).all(): X_c[c] = np.log1p(X_c[c] + self.eps)
        return X_c

class Winsorizer(BaseEstimator, TransformerMixin):
    def __init__(self, l=0.005, u=0.995): self.l, self.u, self.b = l, u, {}
    def fit(self, X, y=None):
        for c in X.columns: self.b[c] = (X[c].quantile(self.l), X[c].quantile(self.u))
        return self
    def transform(self, X):
        X_c = X.copy()
        for c in X_c.columns: X_c[c] = X_c[c].clip(*self.b[c])
        return X_c

### 🛠️ 전처리 파이프라인 구축 및 이상치 제어 실험

모델 학습을 위한 **표준 전처리 파이프라인**을 정의하고, **이상치 제어(Winsorizer)** 효과를 검증했습니다.

1.  **파이프라인 구성**:
    - `결측치/무한대 처리` $\rightarrow$ `로그 변환(왜도 보정)` $\rightarrow$ `Robust Scaling` $\rightarrow$ `SMOTE(옵션)` 순으로 데이터 정제.
    
2.  **Winsorizer 도입 실험**:
    - **가설**: 재무 데이터의 극단적 이상치(Outlier)를 상/하위 0.5%로 제한(Clipping)하면 성능이 오를 것이다.
    - **결과**: 적용 시 **PR-AUC가 하락 (0.1016 $\rightarrow$ 0.0989)** 함을 확인.
    - **결정**: **Winsorizer 미적용 (False)**. 이상치에도 부도 징후 정보가 포함되어 있거나, `RobustScaler`와 `Log변환`만으로도 충분히 제어됨을 시사.

In [5]:
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()

def create_pipeline(clf, wins=False, resamp=None):
    s = [
        ('inf', InfiniteHandler()), 
        ('imp', SimpleImputer(strategy='median').set_output(transform='pandas')), 
        ('log', LogTransformer())
    ]
    if wins: s.append(('wins', Winsorizer()))
    s.append(('scaler', RobustScaler()))
    s.append(('resamp', SMOTE(sampling_strategy=0.2, random_state=RANDOM_STATE) if resamp else 'passthrough'))
    s.append(('clf', clf))
    return ImbPipeline(s)

lgbm_t = lgb.LGBMClassifier(n_estimators=100, max_depth=5, learning_rate=0.05, scale_pos_weight=scale_pos_weight, random_state=RANDOM_STATE, verbose=-1)
print('Winsorizer 실험...')
wins_r = {}
for w in [False, True]:
    p = create_pipeline(lgbm_t, wins=w)
    p.fit(X_train, y_train)
    pr = average_precision_score(y_val, p.predict_proba(X_val)[:, 1])
    wins_r[w] = pr
    print(f'Wins={w}: PR-AUC={pr:.4f}')
USE_WINSORIZER = wins_r[True] > wins_r[False]
print(f'✅ Winsorizer={USE_WINSORIZER}')

Winsorizer 실험...
Wins=False: PR-AUC=0.1016
Wins=True: PR-AUC=0.0989
✅ Winsorizer=False


## 3. 리샘플링 전략 (SMOTE vs Class Weight)

### ⚖️ 불균형 데이터 처리 전략: SMOTE vs Class Weight

1:66의 극심한 데이터 불균형을 해소하기 위해 **데이터 증강(Resampling)**과 **가중치 조정(Cost-sensitive Learning)** 방식을 비교 실험했습니다.

1.  **실험 설계**:
    - **Resampling**: SMOTE 계열 알고리즘을 통해 소수 클래스(부도) 데이터를 인위적으로 생성하여 학습.
    - **Class Weight**: 데이터 생성 없이, 부도 데이터 오분류 시 모델에 더 큰 벌점(Penalty)을 부과하여 학습.

2.  **실험 결과 및 결정**:
    - **SMOTE (0.1230)** > Class Weight (0.1016)
    - 데이터를 직접 증강하는 방식이 모델의 결정 경계를 더 명확하게 형성함이 입증되어, **최종적으로 SMOTE를 채택**했습니다.

In [6]:
lgbm_b = lgb.LGBMClassifier(n_estimators=100, max_depth=5, learning_rate=0.05, random_state=RANDOM_STATE, verbose=-1)
strat_a = {}
for v in ['smote', 'borderline', 'smote_tomek']:
    p = create_pipeline(lgbm_b, wins=USE_WINSORIZER, resamp=v)
    p.fit(X_train, y_train)
    pr = average_precision_score(y_val, p.predict_proba(X_val)[:, 1])
    strat_a[v] = pr
    print(f'{v}: {pr:.4f}')

lgbm_w = lgb.LGBMClassifier(n_estimators=100, max_depth=5, learning_rate=0.05, scale_pos_weight=scale_pos_weight, random_state=RANDOM_STATE, verbose=-1)
p_b = create_pipeline(lgbm_w, wins=USE_WINSORIZER)
p_b.fit(X_train, y_train)
strat_b_pr = average_precision_score(y_val, p_b.predict_proba(X_val)[:, 1])
print(f'Class Weight: {strat_b_pr:.4f}')

best_smote = max(strat_a, key=strat_a.get)
if strat_b_pr > strat_a[best_smote]:
    selected_strategy = 'Class Weight'
    selected_resampler = None
else:
    selected_strategy = f'SMOTE ({best_smote})'
    selected_resampler = best_smote
print(f'✅ 선택: {selected_strategy}')

smote: 0.1230
borderline: 0.1230
smote_tomek: 0.1230
Class Weight: 0.1016
✅ 선택: SMOTE (smote)


## 4. ⭐ AutoML (Priority 1 - n_iter=200)

### 🤖 AutoML: 최적 알고리즘 및 파라미터 탐색

다양한 알고리즘(Boosting, Bagging, Linear)에 대해 하이퍼파라미터 튜닝을 수행하여 최적의 모델을 발굴했습니다.

1.  **모델 다양성 확보**:
    - **Boosting**: LightGBM, XGBoost, CatBoost (강력한 예측력)
    - **Bagging**: RandomForest, ExtraTrees (과적합 방지 및 안정성)
    - **Linear**: LogisticRegression (Baseline 및 설명력)

2.  **최적화 전략**:
    - `RandomizedSearchCV`를 통해 모델별 핵심 파라미터를 효율적으로 탐색.
    - 불균형 데이터 처리를 위해 모델 특성에 맞는 가중치(`scale_pos_weight`, `class_weight`)를 동적으로 할당.

3.  **실험 결과**:
    - **CatBoost (0.1607)** 가 가장 우수한 성능을 기록하여 최종 모델 후보 1순위로 선정.

In [ ]:

# 1. 모델별 하이퍼파라미터 그리드 정의 (공통 파라미터 제거하고 개별 정의)
# -------------------------------------------------------------------------

# Boosting 계열 (scale_pos_weight 사용)
lgbm_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'num_leaves': [31, 63, 127],
    'min_child_samples': [20, 50, 100]
}

xgb_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.8, 0.9, 1.0]
}

cb_grid = {
    'iterations': [100, 200, 300],
    'depth': [4, 6, 8],
    'learning_rate': [0.01, 0.05, 0.1],
    'l2_leaf_reg': [1, 3, 5]
}

# Bagging & Linear 계열 (class_weight 사용)
rf_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

lr_grid = {
    'C': [0.01, 0.1, 1, 10],
    'penalty': ['l2'],
    'solver': ['liblinear']
}

# 2. 가중치(Weight) 동적 할당 로직
# -------------------------------------------------------------------------
if selected_resampler is None:
    # SMOTE 미사용 시: 불균형 해소를 위한 강한 가중치 적용
    # Boosting 계열용
    boost_weights = [scale_pos_weight, scale_pos_weight * 1.5]
    lgbm_grid['scale_pos_weight'] = boost_weights
    xgb_grid['scale_pos_weight'] = boost_weights
    cb_grid['scale_pos_weight'] = boost_weights
    
    # RF, LR 계열용
    rf_grid['class_weight'] = ['balanced', 'balanced_subsample']
    lr_grid['class_weight'] = ['balanced']
else:
    # SMOTE 사용 시: 가중치 1 (또는 None)
    # Boosting 계열용
    lgbm_grid['scale_pos_weight'] = [1]
    xgb_grid['scale_pos_weight'] = [1]
    cb_grid['scale_pos_weight'] = [1]
    
    # RF, LR 계열용
    rf_grid['class_weight'] = [None]
    lr_grid['class_weight'] = [None]

# 3. 모델 정의
# -------------------------------------------------------------------------
models = {
    'LightGBM': (lgb.LGBMClassifier(random_state=RANDOM_STATE, verbose=-1), lgbm_grid),
    'XGBoost': (xgb.XGBClassifier(random_state=RANDOM_STATE, eval_metric='logloss'), xgb_grid),
    'CatBoost': (CatBoostClassifier(random_state=RANDOM_STATE, verbose=0), cb_grid),
    'RandomForest': (RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=-1), rf_grid),
    'ExtraTrees': (ExtraTreesClassifier(random_state=RANDOM_STATE, n_jobs=-1), rf_grid),
    'LogisticRegression': (LogisticRegression(random_state=RANDOM_STATE, max_iter=1000), lr_grid)
}

tuning_results = {}
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
scorer = 'average_precision' 

print(f'AutoML 시작 (n_iter=30, Models={len(models)}개)...') # n_iter 30 정도로 줄여서 테스트 추천

# 4. 학습 루프 (tqdm 적용)
# -------------------------------------------------------------------------
for name, (model, grid) in tqdm(models.items(), desc="Model Tuning Progress"):
    # 파이프라인 생성
    pipe = create_pipeline(model, wins=USE_WINSORIZER, resamp=selected_resampler)
    
    # 파라미터 이름 매핑 (clf__ 접두어 추가)
    pipe_grid = {f'clf__{k}': v for k, v in grid.items()}
    
    try:
        # n_iter=200은 너무 오래 걸릴 수 있으므로 테스트 시엔 줄여보세요 (예: 30~50)
        search = RandomizedSearchCV(pipe, pipe_grid, n_iter=50, scoring=scorer, cv=cv, n_jobs=-1, random_state=RANDOM_STATE, verbose=0)
        search.fit(X_train, y_train)
        
        tuning_results[name] = {
            'best_estimator': search.best_estimator_, 
            'best_cv_score': search.best_score_, 
            'best_params': search.best_params_
        }
        tqdm.write(f'✅ {name}: CV PR-AUC={search.best_score_:.4f}')
        
    except Exception as e:
        tqdm.write(f'❌ {name} 학습 중 오류 발생: {e}')
        # 에러 상세 내용을 보고 싶다면 아래 주석 해제
        # print(e)
        continue

print('🎉 AutoML 완료')

AutoML 시작 (n_iter=30, Models=6개)...


Model Tuning Progress:   0%|          | 0/6 [00:00<?, ?it/s]

✅ LightGBM: CV PR-AUC=0.1238
✅ XGBoost: CV PR-AUC=0.1516
✅ CatBoost: CV PR-AUC=0.1607
✅ RandomForest: CV PR-AUC=0.1559
✅ ExtraTrees: CV PR-AUC=0.1250
✅ LogisticRegression: CV PR-AUC=0.0470
🎉 AutoML 완료


## 5. ⭐ 모델 선택 (Priority 1 - Ensemble + Statistical Test)

### 🏅 최종 모델 선정: Validation 성능 비교

AutoML을 통해 튜닝된 각 모델(LGBM, XGBoost, CatBoost 등)을 **검증 데이터셋(Validation Set)**으로 최종 평가하여 최적의 단일 모델을 선정했습니다.

1.  **평가 지표**: `PR-AUC (Precision-Recall AUC)`
    - 불균형 데이터(1:66)에서 모델의 실질적인 탐지 능력을 가장 잘 대변하는 지표입니다.
    
2.  **선정 결과**:
    - **CatBoost (0.1607)** 가 가장 우수한 성능을 기록하여 **Best Single Model**로 선정되었습니다.
    - 이는 로지스틱 회귀(0.0470) 등 선형 모델 대비 월등한 성능으로, 재무 데이터의 비선형 패턴 학습이 중요함을 시사합니다.

In [8]:
# Validation 평가
val_results = {}
for name, res in tuning_results.items():
    pr = average_precision_score(y_val, res['best_estimator'].predict_proba(X_val)[:, 1])
    val_results[name] = pr
    print(f'{name}: Val PR-AUC={pr:.4f}')

best_single_name = max(val_results, key=val_results.get)
best_single = tuning_results[best_single_name]['best_estimator']
best_single_pr = val_results[best_single_name]
print(f'\nBest Single: {best_single_name} ({best_single_pr:.4f})')

LightGBM: Val PR-AUC=0.1101
XGBoost: Val PR-AUC=0.1118
CatBoost: Val PR-AUC=0.1245
RandomForest: Val PR-AUC=0.1005
ExtraTrees: Val PR-AUC=0.1224
LogisticRegression: Val PR-AUC=0.0268

Best Single: CatBoost (0.1245)


In [22]:
# 📊 모델 성능 비교 시각화 (Bar Chart)
colors = ['#EF553B' if name == best_single_name else '#636EFA' for name in val_results.keys()]

fig = go.Figure(data=[
    go.Bar(
        x=list(val_results.keys()),
        y=list(val_results.values()),
        text=[f'{v:.4f}' for v in val_results.values()],
        textposition='auto',
        marker_color=colors  # 최고 성능 모델 강조 (빨간색)
    )
])

fig.update_layout(
    title=f'<b>Model Comparison (Validation PR-AUC)</b><br>Best: {best_single_name}',
    xaxis_title='Model',
    yaxis_title='PR-AUC Score',
    template='plotly_white',
    height=500
)

fig.show()

In [9]:
# Voting Ensemble (상위 3개)
sorted_models = sorted(val_results.items(), key=lambda x: x[1], reverse=True)[:3]
estimators = [(name, tuning_results[name]['best_estimator']) for name, _ in sorted_models]
voting = VotingClassifier(estimators=estimators, voting='soft')
voting.fit(X_train, y_train)
voting_pr = average_precision_score(y_val, voting.predict_proba(X_val)[:, 1])
print(f'Voting Ensemble: Val PR-AUC={voting_pr:.4f}')
print(f'\n성능 차이: {voting_pr - best_single_pr:+.4f}')

Voting Ensemble: Val PR-AUC=0.1263

성능 차이: +0.0018


In [23]:
# 📊 앙상블 포함 최종 성능 비교 시각화
import plotly.graph_objects as go

# 1. 결과 통합 및 정렬
final_comparison = val_results.copy()
final_comparison['Voting Ensemble'] = voting_pr
sorted_comparison = dict(sorted(final_comparison.items(), key=lambda item: item[1], reverse=True))

# 2. 색상 설정 (Ensemble: 초록, Best Single: 빨강, 나머지: 파랑)
colors = []
for name in sorted_comparison.keys():
    if name == 'Voting Ensemble':
        colors.append('#00CC96') # 강조 (초록)
    elif name == best_single_name:
        colors.append('#EF553B') # 기존 1위 (빨강)
    else:
        colors.append('#636EFA') # 기본 (파랑)

# 3. 차트 생성
fig = go.Figure(data=[
    go.Bar(
        x=list(sorted_comparison.keys()),
        y=list(sorted_comparison.values()),
        text=[f'{v:.4f}' for v in sorted_comparison.values()],
        textposition='auto',
        marker_color=colors
    )
])

# 4. 레이아웃 설정
diff = voting_pr - best_single_pr
diff_text = f"Ensemble Effect: {diff:+.4f} ({'Improved' if diff > 0 else 'No Improvement'})"

fig.update_layout(
    title=f'<b>Model Performance Comparison (w/ Ensemble)</b><br><span style="font-size:12px;color:gray">{diff_text}</span>',
    xaxis_title='Model',
    yaxis_title='PR-AUC Score',
    template='plotly_white',
    height=500,
    yaxis=dict(range=[0, max(sorted_comparison.values()) * 1.1]) # y축 여유 공간 확보
)

fig.show()

### ⚖️ 최종 모델 선정: 앙상블의 실효성 검증

가장 성능이 좋은 단일 모델(**CatBoost**)과 앙상블 모델(**Voting**) 간의 성능 차이가 통계적으로 유의미한지 **Wilcoxon Test**로 검증했습니다.

1.  **검정 결과**:
    - **p-value: 0.0625** ($\ge$ 0.05)
    - 통계적 유의수준 5% 하에서 두 모델 간의 성능 차이는 유의미하지 않음(기각 실패).

2.  **최종 의사결정 (오컴의 면도날)**:
    - **✅ 최종 모델: CatBoost (Best Single)**
    - 앙상블 도입으로 인한 복잡도 증가 대비 성능 이득이 확실하지 않으므로, **설명력과 운영 효율성이 높은 단일 모델**을 최종 채택했습니다.

In [10]:
# Statistical Significance Test
print('\n통계적 유의성 검정 (Wilcoxon)...')
cv_single = cross_val_score(best_single, X_train, y_train, cv=5, scoring=scorer)
cv_voting = cross_val_score(voting, X_train, y_train, cv=5, scoring=scorer)
stat, pval = wilcoxon(cv_voting, cv_single)
print(f'p-value: {pval:.4f}')

if voting_pr > best_single_pr and pval < 0.05:
    final_model = voting
    final_name = 'VotingEnsemble'
    reason = f'Ensemble이 {voting_pr - best_single_pr:.4f} 더 우수 (p={pval:.4f} < 0.05)'
else:
    final_model = best_single
    final_name = best_single_name
    reason = f'Single 선택 (차이 미미 또는 p={pval:.4f} >= 0.05)'

print(f'\n✅ 최종 모델: {final_name}')
print(f'   이유: {reason}')


통계적 유의성 검정 (Wilcoxon)...
p-value: 0.0625

✅ 최종 모델: CatBoost
   이유: Single 선택 (차이 미미 또는 p=0.0625 >= 0.05)


## 6. ⭐ 임계값 최적화 (Priority 2 - Validation + CV)

### 🎯 최적 임계값(Threshold) 산출: F2-Score vs Recall

부도 예측 모델의 실효성을 높이기 위해, 기본 임계값(0.5) 대신 **비즈니스 목적에 부합하는 최적 임계값**을 탐색했습니다.

1.  **임계값 설정 기준**:
    - **Max F2-Score**: 재현율(Recall)에 정밀도(Precision)보다 2배 가중치를 두어, 부도 미탐지 비용을 줄이는 최적점.
    - **Recall 80%**: "전체 부도 기업의 최소 80%는 반드시 조기 발견해야 한다"는 안전장치(Safety Net) 기준.

2.  **산출 결과**:
    - **F2 최적점 (0.1755)**: 모델의 종합적인 성능 효율성이 가장 높은 지점.
    - **Recall 80% (0.0468)**: 더 많은 잠재 부도 기업을 포착하기 위해 정밀도 손실을 감수한 보수적 기준.

In [11]:
y_val_prob_final = final_model.predict_proba(X_val)[:, 1]
prec_v, rec_v, thr_v = precision_recall_curve(y_val, y_val_prob_final)

# F2-Score 최적
beta = 2
f2_scores = (1 + beta**2) * (prec_v[:-1] * rec_v[:-1]) / (beta**2 * prec_v[:-1] + rec_v[:-1] + 1e-10)
f2_opt_idx = np.argmax(f2_scores)
f2_opt_thr = thr_v[f2_opt_idx]
print(f'F2 최적 임계값 (Val): {f2_opt_thr:.4f}')

# Recall 80%
idx_80 = np.where(rec_v[:-1] >= 0.80)[0]
if len(idx_80) > 0:
    r80_idx = idx_80[np.argmax(prec_v[:-1][idx_80])]
    r80_thr = thr_v[r80_idx]
    print(f'Recall 80% 임계값 (Val): {r80_thr:.4f}')
else:
    r80_thr = f2_opt_thr
    print('Recall 80% 불가, F2 사용')

F2 최적 임계값 (Val): 0.1755
Recall 80% 임계값 (Val): 0.0468


In [ ]:
prec_v, rec_v, thr_v = precision_recall_curve(y_val, y_val_prob_final)

# F2-Score 계산
beta = 2
f2_scores = (1 + beta**2) * (prec_v[:-1] * rec_v[:-1]) / (beta**2 * prec_v[:-1] + rec_v[:-1] + 1e-10)
f2_opt_idx = np.argmax(f2_scores)
f2_opt_thr = thr_v[f2_opt_idx]
max_f2_score = f2_scores[f2_opt_idx]

# Recall 80% 지점 찾기
idx_80 = np.where(rec_v[:-1] >= 0.80)[0]
r80_thr = thr_v[idx_80[np.argmax(prec_v[:-1][idx_80])]] if len(idx_80) > 0 else f2_opt_thr

# -------------------------------------------------------
# 시각화 시작
# -------------------------------------------------------
fig = go.Figure()

# 1. Precision Curve (파란 점선)
fig.add_trace(go.Scatter(
    x=thr_v, 
    y=prec_v[:-1], 
    mode='lines', 
    name='Precision',
    line=dict(color='#636EFA', dash='dash')
))

# 2. Recall Curve (빨간 점선)
fig.add_trace(go.Scatter(
    x=thr_v, 
    y=rec_v[:-1], 
    mode='lines', 
    name='Recall',
    line=dict(color='#EF553B', dash='dash')
))

# 3. F2-Score Curve (초록 실선 - 메인)
fig.add_trace(go.Scatter(
    x=thr_v, 
    y=f2_scores, 
    mode='lines', 
    name='F2-Score',
    line=dict(color='#00CC96', width=3)
))

# 4. 최적 임계값 표시 (수직선 & 주석)
# (1) Max F2 Threshold
fig.add_vline(x=f2_opt_thr, line_width=1, line_dash="dot", line_color="black")

# [수정] 텍스트 위치를 y축보다 조금 더 높게 잡고(yshift), 화살표를 사용하여 선과 겹침 방지
fig.add_annotation(
    x=f2_opt_thr, 
    y=max_f2_score,
    text=f"<b>Max F2: {max_f2_score:.3f}</b><br>(Thr: {f2_opt_thr:.4f})",
    showarrow=True, 
    arrowhead=2,
    ax=0,           # x축 방향 이동 (0이면 수직 위)
    ay=-40,         # y축 방향 이동 (-40이면 위로 40픽셀)
    bgcolor="rgba(255, 255, 255, 0.8)", # 텍스트 배경을 반투명 흰색으로 하여 선 가림 방지
    bordercolor="black",
    borderwidth=1
)

# (2) Recall 80% Threshold
if len(idx_80) > 0:
    fig.add_vline(x=r80_thr, line_width=1, line_dash="dot", line_color="orange")
    fig.add_annotation(
        x=r80_thr, 
        y=0.8,
        text=f"<b>Recall 80%</b><br>(Thr: {r80_thr:.4f})",
        showarrow=True, 
        arrowhead=1,
        ax=50,   # 오른쪽으로 이동
        ay=-30,  # 위쪽으로 이동
        bgcolor="rgba(255, 255, 255, 0.8)"
    )

# 레이아웃 설정
fig.update_layout(
    title='<b>Threshold Optimization Curves</b><br><span style="font-size:12px;color:gray">Precision, Recall, and F2-Score vs Threshold</span>',
    xaxis_title='Threshold (Probability)',
    yaxis_title='Score',
    yaxis=dict(range=[0, 1.1]), # y축 범위를 살짝 넓혀서 상단 텍스트 공간 확보
    template='plotly_white',
    height=550,
    
    # [수정] 범례(Legend)를 그래프 오른쪽 바깥으로 이동
    legend=dict(
        x=1.02,      # x=1이 그래프 끝이므로 1.02는 바깥쪽
        y=1,         # y=1은 상단
        xanchor='left',
        yanchor='top',
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='Black',
        borderwidth=1
    ),
    margin=dict(r=150) # 오른쪽 여백 확보 (범례 공간)
)

fig.show()

### 6.1 ⭐ CV 기반 임계값 검증 (Priority 2)

### 🛡️ 임계값 교차 검증 및 최종 확정 (Robust Thresholding)

단일 검증 데이터(Validation Set)에만 의존할 경우 발생할 수 있는 과적합 위험을 줄이기 위해, **교차 검증(CV)** 결과를 결합하여 최종 임계값을 확정했습니다.

1.  **검증 전략**:
    - `Validation Set 기준`과 `5-Fold CV 기준`으로 각각 Recall 80% 임계값을 산출.
    - 두 값의 **평균(Average)**을 최종 임계값으로 채택하여, 특정 데이터셋에 치우치지 않는 **일반화된 기준**을 수립.

2.  **최종 결정**:
    - **최종 임계값: 0.0497** (Val 0.0468 + CV 0.0525의 평균)
    - 모델이 예측한 부도 확률이 **4.97%** 이상일 경우, 이를 '부도 위험군'으로 판정하고 관리 대상으로 분류합니다.

In [12]:
# CV 예측
y_train_prob_cv = cross_val_predict(final_model, X_train, y_train, cv=5, method='predict_proba')[:, 1]
prec_cv, rec_cv, thr_cv = precision_recall_curve(y_train, y_train_prob_cv)

f2_cv = (1 + beta**2) * (prec_cv[:-1] * rec_cv[:-1]) / (beta**2 * prec_cv[:-1] + rec_cv[:-1] + 1e-10)
f2_cv_thr = thr_cv[np.argmax(f2_cv)]

idx_cv_80 = np.where(rec_cv[:-1] >= 0.80)[0]
if len(idx_cv_80) > 0:
    r80_cv_thr = thr_cv[idx_cv_80[np.argmax(prec_cv[:-1][idx_cv_80])]]
else:
    r80_cv_thr = f2_cv_thr

print(f'CV F2 임계값: {f2_cv_thr:.4f}')
print(f'CV Recall 80% 임계값: {r80_cv_thr:.4f}')

# 평균
selected_threshold = (r80_thr + r80_cv_thr) / 2
print(f'\n✅ 최종 임계값: {selected_threshold:.4f} (Val+CV 평균)')

CV F2 임계값: 0.1907
CV Recall 80% 임계값: 0.0525

✅ 최종 임계값: 0.0497 (Val+CV 평균)


In [26]:
# 1. CV 예측 및 지표 계산 (기존 로직)
# -------------------------------------------------------
y_train_prob_cv = cross_val_predict(final_model, X_train, y_train, cv=5, method='predict_proba')[:, 1]
prec_cv, rec_cv, thr_cv = precision_recall_curve(y_train, y_train_prob_cv)

# F2-Score 계산
beta = 2
f2_cv = (1 + beta**2) * (prec_cv[:-1] * rec_cv[:-1]) / (beta**2 * prec_cv[:-1] + rec_cv[:-1] + 1e-10)
f2_cv_idx = np.argmax(f2_cv)
f2_cv_thr = thr_cv[f2_cv_idx]
max_f2_cv_score = f2_cv[f2_cv_idx]

# Recall 80% 지점 찾기
idx_cv_80 = np.where(rec_cv[:-1] >= 0.80)[0]
if len(idx_cv_80) > 0:
    r80_cv_thr = thr_cv[idx_cv_80[np.argmax(prec_cv[:-1][idx_cv_80])]]
else:
    r80_cv_thr = f2_cv_thr

# 최종 임계값 (Val과 CV의 평균)
# r80_thr는 이전 단계(Validation)에서 정의되어 있어야 합니다.
selected_threshold = (r80_thr + r80_cv_thr) / 2

print(f'CV F2 최적 임계값: {f2_cv_thr:.4f}')
print(f'CV Recall 80% 임계값: {r80_cv_thr:.4f}')
print(f'✅ 최종 임계값 (Val+CV 평균): {selected_threshold:.4f}')

# 2. 시각화
# -------------------------------------------------------
fig = go.Figure()

# (1) CV Curves
fig.add_trace(go.Scatter(x=thr_cv, y=prec_cv[:-1], mode='lines', name='CV Precision', line=dict(color='#636EFA', dash='dash')))
fig.add_trace(go.Scatter(x=thr_cv, y=rec_cv[:-1], mode='lines', name='CV Recall', line=dict(color='#EF553B', dash='dash')))
fig.add_trace(go.Scatter(x=thr_cv, y=f2_cv, mode='lines', name='CV F2-Score', line=dict(color='#00CC96', width=3)))

# (2) Critical Points Annotations
# CV Max F2
fig.add_vline(x=f2_cv_thr, line_width=1, line_dash="dot", line_color="black")
fig.add_annotation(
    x=f2_cv_thr, y=max_f2_cv_score,
    text=f"<b>CV Max F2</b><br>{f2_cv_thr:.4f}",
    showarrow=True, arrowhead=2,
    ax=0, ay=-40,
    bgcolor="rgba(255, 255, 255, 0.8)", bordercolor="black"
)

# CV Recall 80%
fig.add_vline(x=r80_cv_thr, line_width=1, line_dash="dot", line_color="orange")
fig.add_annotation(
    x=r80_cv_thr, y=0.8,
    text=f"<b>CV Rec 80%</b><br>{r80_cv_thr:.4f}",
    showarrow=True, arrowhead=1,
    ax=50, ay=-30,
    bgcolor="rgba(255, 255, 255, 0.8)"
)

# (3) Final Selected Threshold (보라색 굵은 선)
fig.add_vline(x=selected_threshold, line_width=3, line_color="#AB63FA")
fig.add_annotation(
    x=selected_threshold, y=0.5,
    text=f"<b>Final Selection</b><br>(Avg: {selected_threshold:.4f})",
    showarrow=True, arrowhead=2,
    ax=-70, ay=0,
    bgcolor="rgba(255, 255, 255, 0.9)", bordercolor="#AB63FA", borderwidth=2
)

# 레이아웃 설정
fig.update_layout(
    title='<b>Cross-Validation Threshold Check</b><br><span style="font-size:12px;color:gray">CV Precision, Recall, F2 & Final Decision</span>',
    xaxis_title='Threshold',
    yaxis_title='Score',
    yaxis=dict(range=[0, 1.1]),
    template='plotly_white',
    height=550,
    legend=dict(x=1.02, y=1, xanchor='left', yanchor='top', bgcolor='rgba(255, 255, 255, 0.5)'),
    margin=dict(r=150)
)

fig.show()

CV F2 최적 임계값: 0.1907
CV Recall 80% 임계값: 0.0525
✅ 최종 임계값 (Val+CV 평균): 0.0497


## 7. Traffic Light 시스템

### 🚦 Traffic Light 리스크 등급화 (3단계 관리 체계)

모델의 예측 확률을 기반으로 기업을 3가지 리스크 등급으로 분류하여, **실무적 대응 우선순위**를 수립했습니다.

1.  **등급 기준 (Thresholding)**:
    - 🔴 **Red (고위험군)**: 부도 기업의 **80%**를 포착하는 구간 (Threshold $\ge$ 0.0468). 즉시 정밀 진단 필요.
    - 🟡 **Yellow (잠재위험군)**: 부도 기업의 **95%**까지 커버하는 안전장치 구간 (Threshold $\ge$ 0.0168). 모니터링 강화.
    - 🟢 **Green (정상군)**: 나머지 하위 구간. 리스크가 현저히 낮음.

2.  **검증 결과 (Validation Set)**:
    - **Red 등급**: 전체의 약 29%인 2,879개 기업을 선별하여, 실제 부도 기업 123개를 집중 포착했습니다.
    - **Green 등급**: 3,903개 기업 중 실제 부도는 단 7건에 불과하여, 모델의 **안전성(Safety)**을 확인했습니다.

In [13]:
# Red: Recall 80%
red_threshold = r80_thr

# Yellow: Recall 95%
idx_95 = np.where(rec_v[:-1] >= 0.95)[0]
if len(idx_95) > 0:
    yellow_threshold = thr_v[idx_95[np.argmax(prec_v[:-1][idx_95])]]
else:
    yellow_threshold = thr_v[-1] if len(thr_v) > 0 else 0.01

print(f'Red >= {red_threshold:.4f}')
print(f'Yellow >= {yellow_threshold:.4f}')

def assign_traffic(prob, red, yellow):
    if prob >= red: return 'Red'
    elif prob >= yellow: return 'Yellow'
    else: return 'Green'

# Val 평가
traffic_val = pd.Series(y_val_prob_final).apply(lambda x: assign_traffic(x, red_threshold, yellow_threshold))
for grade in ['Red', 'Yellow', 'Green']:
    mask = (traffic_val == grade)
    cnt = mask.sum()
    bk = y_val.values[mask].sum()
    print(f'{grade}: {cnt} 기업, {bk} 부도')

Red >= 0.0468
Yellow >= 0.0168
Red: 2879 기업, 123 부도
Yellow: 3239 기업, 22 부도
Green: 3903 기업, 7 부도


In [27]:
# 1. 데이터 집계 (기존 로직 활용)
# -------------------------------------------------------
# Traffic Light 할당
traffic_val = pd.Series(y_val_prob_final).apply(lambda x: assign_traffic(x, red_threshold, yellow_threshold))

# 등급별 통계 계산
grades = ['Red', 'Yellow', 'Green']
grade_counts = []
grade_bks = []
grade_rates = []

for grade in grades:
    mask = (traffic_val == grade)
    cnt = mask.sum()
    bk = y_val.values[mask].sum()
    rate = bk / cnt if cnt > 0 else 0
    
    grade_counts.append(cnt)
    grade_bks.append(bk)
    grade_rates.append(rate)

# 2. 시각화 (Bar Chart with Annotations)
# -------------------------------------------------------
# 색상 정의 (Red, Yellow, Green)
colors = ['#EF553B', '#FECB52', '#00CC96']

fig = go.Figure()

# 막대 그래프 (기업 수)
fig.add_trace(go.Bar(
    x=grades,
    y=grade_counts,
    marker_color=colors,
    text=[f"<b>{bk}</b> Bankruptcies<br>({rate:.1%})" for bk, rate in zip(grade_bks, grade_rates)],
    textposition='auto',
    name='Total Companies'
))

# 레이아웃 설정
fig.update_layout(
    title=f'<b>Traffic Light Risk Distribution</b><br><span style="font-size:12px;color:gray">Red(Recall 80%) >= {red_threshold:.4f}, Yellow(Recall 95%) >= {yellow_threshold:.4f}</span>',
    xaxis_title='Risk Grade',
    yaxis_title='Number of Companies',
    template='plotly_white',
    height=500,
    showlegend=False
)

# 부도율(Risk Rate)을 별도 텍스트로 상단에 표시 (옵션)
for i, (cnt, rate) in enumerate(zip(grade_counts, grade_rates)):
    fig.add_annotation(
        x=grades[i], y=cnt,
        text=f"Total: {cnt:,}",
        showarrow=False,
        yshift=10,
        font=dict(color="black")
    )

fig.show()

## 8. ⭐ Test Set 최종 평가 (Priority 2 - 시각화)

### 8.1 Test Set 예측

### 🎯 최종 관문: Test Set 성능 평가 (Generalization Check)

학습 및 검증 과정에 전혀 관여하지 않은 **Test Set(완전한 미지 데이터)**을 통해 모델의 **일반화 성능**을 최종 검증했습니다.

1.  **조기 경보 능력 (Recall) 입증**:
    - **Recall 86.84%**: 실제 부도 기업 152개 중 **132개**를 사전에 포착하는 데 성공했습니다.
    - 놓친 기업(FN)은 단 **20개**에 불과하여, 금융 리스크 관리의 핵심인 '안전장치' 역할을 충실히 수행함을 확인했습니다.

2.  **모델 효율성 (PR-AUC)**:
    - **PR-AUC 0.1602**: 무작위 예측(Baseline 약 1.5%) 대비 **10배 이상의 예측 효율**을 달성했습니다.
    - **Trade-off**: 높은 Recall(87%)을 위해 낮은 Precision(5%)을 감수하는 전략은, 부도 미탐지 비용이 오탐지 비용보다 훨씬 큰 도메인 특성상 합리적인 결과입니다.

In [14]:
y_test_prob = final_model.predict_proba(X_test)[:, 1]
y_test_pred = (y_test_prob >= selected_threshold).astype(int)

test_pr_auc = average_precision_score(y_test, y_test_prob)
test_roc_auc = roc_auc_score(y_test, y_test_prob)
test_f2 = fbeta_score(y_test, y_test_pred, beta=2)

tn, fp, fn, tp = confusion_matrix(y_test, y_test_pred).ravel()
test_recall = tp / (tp + fn)
test_prec = tp / (tp + fp)

print('='*70)
print('🎯 Test Set 최종 평가')
print('='*70)
print(f'PR-AUC: {test_pr_auc:.4f}')
print(f'ROC-AUC: {test_roc_auc:.4f}')
print(f'F2-Score: {test_f2:.4f}')
print(f'Precision: {test_prec:.2%}')
print(f'Recall: {test_recall:.2%}')
print(f'\nConfusion Matrix:')
print(f'  TN: {tn:,}  |  FP: {fp:,}')
print(f'  FN: {fn:,}  |  TP: {tp:,}')
print('='*70)

🎯 Test Set 최종 평가
PR-AUC: 0.1602
ROC-AUC: 0.8847
F2-Score: 0.2046
Precision: 5.04%
Recall: 86.84%

Confusion Matrix:
  TN: 7,383  |  FP: 2,486
  FN: 20  |  TP: 132


In [ ]:
# 1. 데이터 준비
metrics = ['PR-AUC', 'ROC-AUC', 'F2-Score', 'Precision', 'Recall']
values = [test_pr_auc, test_roc_auc, test_f2, test_prec, test_recall]
colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A']

# 2. 시각화 (Horizontal Bar Chart)
fig = go.Figure(go.Bar(
    x=values,
    y=metrics,
    orientation='h',
    text=[f'{v:.4f}' for v in values],  # 막대 위에 값 표시
    textposition='auto',
    marker_color=colors
))

# 3. 레이아웃 설정
fig.update_layout(
    title='<b>Test Set Final Performance</b>',
    xaxis_title='Score',
    xaxis=dict(range=[0, 1.05]), # 0~1 범위 고정
    yaxis=dict(autorange="reversed"), # 위에서부터 순서대로 표시
    template='plotly_white',
    height=400,
    showlegend=False
)

fig.show()

### 8.2 ⭐ PR-AUC Curve 시각화 (Priority 2)

In [15]:
prec_test, rec_test, thr_test = precision_recall_curve(y_test, y_test_prob)

fig = go.Figure()
fig.add_trace(go.Scatter(x=rec_test, y=prec_test, mode='lines', name=f'PR-AUC={test_pr_auc:.4f}', line=dict(width=2)))
fig.add_trace(go.Scatter(x=[0, 1], y=[y_test.mean(), y_test.mean()], mode='lines', name='Baseline', line=dict(dash='dash')))
fig.update_layout(title='Precision-Recall Curve (Test Set)', xaxis_title='Recall', yaxis_title='Precision', height=500)
fig.show()

### 8.3 ⭐ Confusion Matrix 시각화 (Priority 2)

In [16]:
cm = confusion_matrix(y_test, y_test_pred)
fig = go.Figure(data=go.Heatmap(z=cm, x=['Pred 0', 'Pred 1'], y=['True 0', 'True 1'], text=cm, texttemplate='%{text}', colorscale='Blues'))
fig.update_layout(title=f'Confusion Matrix (Threshold={selected_threshold:.4f})', height=400)
fig.show()

## 9. ⭐ 모델 진단 (Priority 3)

### 9.1 ⭐ Calibration Check (Priority 3)

In [17]:
prob_true, prob_pred = calibration_curve(y_val, y_val_prob_final, n_bins=10, strategy='quantile')

fig = go.Figure()
fig.add_trace(go.Scatter(x=prob_pred, y=prob_true, mode='markers+lines', name='Model', marker=dict(size=10)))
fig.add_trace(go.Scatter(x=[0,1], y=[0,1], mode='lines', name='Perfect', line=dict(dash='dash')))
fig.update_layout(title='Calibration Curve (Validation Set)', xaxis_title='Predicted Probability', yaxis_title='True Probability', height=500)
fig.show()
print('\n✅ Calibration: 대각선에 가까울수록 확률 예측 정확')


✅ Calibration: 대각선에 가까울수록 확률 예측 정확


### 9.2 ⭐ Learning Curve (Priority 3)

In [18]:
train_sizes, train_scores, val_scores = learning_curve(
    final_model, X_train, y_train,
    cv=5, scoring=scorer,
    train_sizes=np.linspace(0.1, 1.0, 10),
    n_jobs=-1, random_state=RANDOM_STATE
)

fig = go.Figure()
fig.add_trace(go.Scatter(x=train_sizes, y=train_scores.mean(axis=1), mode='lines+markers', name='Train', error_y=dict(array=train_scores.std(axis=1))))
fig.add_trace(go.Scatter(x=train_sizes, y=val_scores.mean(axis=1), mode='lines+markers', name='CV', error_y=dict(array=val_scores.std(axis=1))))
fig.update_layout(title='Learning Curve', xaxis_title='Training Size', yaxis_title='PR-AUC', height=500)
fig.show()
print('\n✅ Learning Curve: Train과 CV 격차가 작으면 과적합 없음')


✅ Learning Curve: Train과 CV 격차가 작으면 과적합 없음


## 10. ⭐ Cumulative Gains Curve (Priority 2)

In [19]:
# Test Set Cumulative Gains
df_gains = pd.DataFrame({'prob': y_test_prob, 'y': y_test.values})
df_gains = df_gains.sort_values('prob', ascending=False).reset_index(drop=True)
df_gains['cum_y'] = df_gains['y'].cumsum()
df_gains['pct_captured'] = df_gains['cum_y'] / df_gains['y'].sum()
df_gains['pct_population'] = (df_gains.index + 1) / len(df_gains)

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_gains['pct_population'], y=df_gains['pct_captured'], mode='lines', name='Model', line=dict(width=2)))
fig.add_trace(go.Scatter(x=[0,1], y=[0,1], mode='lines', name='Random', line=dict(dash='dash')))
fig.update_layout(title='Cumulative Gains Curve (Test Set)', xaxis_title='% Population', yaxis_title='% Bankruptcies Captured', xaxis_tickformat='.0%', yaxis_tickformat='.0%', height=500)
fig.show()

# 상위 10% 포착률
top10_captured = df_gains[df_gains['pct_population'] <= 0.1]['pct_captured'].iloc[-1]
print(f'\n✅ 상위 10% 기업에서 부도의 {top10_captured:.1%} 포착')
print(f'   효율성: Random 대비 {top10_captured/0.1:.1f}배')


✅ 상위 10% 기업에서 부도의 63.2% 포착
   효율성: Random 대비 6.3배


## 11. Feature Importance

### 📊 모델 해석: 주요 변수 중요도 (Feature Importance) 분석

최종 선정된 모델이 어떤 변수에 기반하여 부도를 예측하는지 확인하기 위해 **변수 중요도**를 시각화했습니다.

1.  **분석 방법**:
    - **Tree 기반 모델 (CatBoost 등)**: 불순도(Gini Impurity) 감소 기여도를 기준으로 상위 15개 핵심 변수를 추출하여 `Viridis` 컬러 스케일로 표현.
    - **선형 모델 (Logistic Regression)**: 각 변수의 회귀 계수(Coefficient)를 추출하여, 부도 위험을 높이는 변수(**Red**)와 낮추는 변수(**Blue**)를 명확히 대비.

2.  **분석 목적**:
    - 모델의 **설명 가능성(Explainability)** 확보.
    - 재무적 상식과 모델의 학습 결과가 일치하는지 검증 (예: 유동비율이 낮을수록 위험한가?).

In [ ]:
if final_name == 'VotingEnsemble':
    # Ensemble이면 첫 번째 모델 사용
    clf = final_model.estimators_[0].named_steps['clf']
else:
    clf = final_model.named_steps['clf']

# 1. Tree 기반 모델 Feature Importance (Viridis 스케일 적용)
if hasattr(clf, 'feature_importances_'):
    imp = clf.feature_importances_
    feat_imp = pd.DataFrame({'Feature': X_train.columns, 'Importance': imp}).sort_values('Importance', ascending=False).head(15)
    
    fig = go.Figure(go.Bar(
        x=feat_imp['Importance'], 
        y=feat_imp['Feature'], 
        orientation='h', 
        marker=dict(
            color=feat_imp['Importance'], 
            colorscale='Viridis' # [수정] Blues보다 명확한 대비를 주는 스케일
        )
    ))
    fig.update_layout(title=f'Top 15 Feature Importance ({final_name})', xaxis_title='Importance', yaxis_title='Feature', height=600, yaxis={'categoryorder':'total ascending'})
    fig.show()
else:
    print('Feature Importance 지원 안 함')

# 2. LogisticRegression 계수 확인 (양수/음수 단색 대비 적용)
if final_name == 'LogisticRegression':
    # 파이프라인에서 분류기(clf) 단계 가져오기
    clf = final_model.named_steps['clf']
    
    # 계수 가져오기
    coefficients = clf.coef_[0]
    features = X_train.columns
    
    # 데이터프레임 생성
    coef_df = pd.DataFrame({'Feature': features, 'Coefficient': coefficients})
    
    # 절대값 기준으로 정렬 (영향력이 큰 순서대로)
    coef_df['Abs_Coef'] = coef_df['Coefficient'].abs()
    coef_df = coef_df.sort_values(by='Abs_Coef', ascending=False).head(20)
    
    # [수정] 0을 기준으로 양수(Red), 음수(Blue) 단색 지정 (명확한 대비)
    colors = ['#EF553B' if c > 0 else '#636EFA' for c in coef_df['Coefficient']]
    
    # 시각화
    fig = go.Figure()
    fig.add_trace(go.Bar(
        y=coef_df['Feature'],
        x=coef_df['Coefficient'],
        orientation='h',
        marker=dict(color=colors) # colorscale 대신 직접 지정한 색상 리스트 사용
    ))
    
    fig.update_layout(
        title=f'Top 20 Coefficients (Impact) - {final_name}<br><span style="font-size:12px;color:gray">Red: Increases Risk (+), Blue: Decreases Risk (-)</span>',
        xaxis_title='Coefficient Value (Log-Odds Impact)',
        yaxis=dict(categoryorder='total ascending'),
        height=600
    )
    fig.show()

## 12. 모델 및 결과 저장

In [21]:
save_files = {
    f'{OUTPUT_PREFIX}_최종모델.pkl': final_model,
    f'{OUTPUT_PREFIX}_임계값.pkl': {'selected': selected_threshold, 'red': red_threshold, 'yellow': yellow_threshold},
    f'{OUTPUT_PREFIX}_결과.pkl': {'model_name': final_name, 'test_pr_auc': test_pr_auc, 'test_recall': test_recall, 'test_f2': test_f2}
}

for fname, data in save_files.items():
    joblib.dump(data, os.path.join(PROCESSED_DIR, fname))
    print(f'✅ {fname}')

print(f'\n저장 위치: {PROCESSED_DIR}')

✅ 발표_Part3_v3_최종모델.pkl
✅ 발표_Part3_v3_임계값.pkl
✅ 발표_Part3_v3_결과.pkl

저장 위치: ../data/processed


## 13. 최종 요약

### ✅ v3 개선사항 및 실험 결과

**Priority 1: 모델 고도화**
- **Ensemble 검증:** Wilcoxon Test 결과 **p-value 0.0625** ($\ge$ 0.05)로 단일 모델과 유의미한 차이 없음.
    - $\rightarrow$ **CatBoost (Single)** 최종 선정 (유지보수 및 설명력 우위).
- **AutoML:** 탐색 횟수 확장 (`n_iter` 30 $\rightarrow$ 50회 수행).

**Priority 2: 최적화 및 시각화**
- **Robust 임계값:** Validation(0.0468)과 CV(0.0525)의 평균인 **0.0497**로 확정.
- **비즈니스 시각화:** Cumulative Gains Curve, Traffic Light Risk 등급, PR Curve 등 완료.

**Priority 3: 전처리 및 진단**
- **Winsorizer 실험:** 적용 시 성능 소폭 하락(0.1016 $\rightarrow$ 0.0989)하여 **미적용(False)** 결정.
- **모델 진단:** Calibration Curve(확률 정합성) 및 Learning Curve(학습 안정성) 양호.

### 🎯 최종 핵심 성과 (Test Set 평가)

✅ **Data Leakage 완전 제거:** 순수 재무 지표 기반의 건전한 모델 구축.
✅ **조기 경보 능력 입증:** 실제 부도 기업의 **86.84% (Recall)** 사전 탐지 성공.
✅ **예측 효율성 확보:** PR-AUC **0.1602** 달성 (Baseline 1.5% 대비 10배 이상).
✅ **운영 신뢰성 확보:** 통계적 검증을 거친 모델과 보수적 임계값(4.97%) 적용.

---

**노트북 완료!** 🎉